In [391]:
## importing required modules

import Bio
import pandas as pd
import csv
import re

import matplotlib.pyplot as plt

from Bio import SeqIO
from Bio.Seq import Seq
from Bio import AlignIO
from Bio.Align.AlignInfo import SummaryInfo

In [392]:
## load in the bio.seqIO index of the sequences
sequences_index = Bio.SeqIO.index("designated_seqs.fasta", "fasta")

In [394]:
## load in the reference sequence
reference_A = SeqIO.read("reference_WUHAN.fasta.txt", "fasta")

In [9]:
lineages_list = list(lineage_designations["lineage"].unique())

lineages = []
number_seq = []

for row in list(lineage_designations["lineage"].unique()): 
    lineages.append(row)
    number_seq.append(len(lineage_designations.loc[lineage_designations["lineage"] == str(row)]))
    # lineages = row
    # number_seqs = len(lineage_designations.loc[lineage_designations["lineage"] == str(row)])

no_of_sequences_per_lineage = pd.DataFrame(
    {'lineage': lineages,
     'no_of_sequences': number_seq})

no_of_sequences_per_lineage.to_csv('sequences_per_lineage.csv')

## save all of the lineage counts to a csv file

In [418]:
## this function gives all of the potential outputs - could be split up into seperate functions, but each part relies on the same basic code so is more efficient to run all at once 

def full_function_mutations(lineage_of_interest, designation_index, reference_sequence): 
    
    sequence_ID_list = []
    
    relevant_rows = {}
    with open("lineages.csv","r") as f:
        reader = csv.DictReader(f)
        for row in reader:
            if row["lineage"] == str(lineage_of_interest): 
                relevant_rows[row["taxon"]] = row
                                
    for row in relevant_rows: 
        sequence_ID_list.append(row)
        
    ### output of list of sequence_IDs for the given lineage
    ### output of list of sequence_IDs for the given lineage
    
    sequences_lineage = Bio.Align.MultipleSeqAlignment([])
    
    for sequence in designation_index: ## for a sequence in the indexed sequences
        if sequence in sequence_ID_list: ## is this sequence in the ID list? - this is needed as there are some missing
            sequences_lineage.append(designation_index[sequence]) ## if it is, add the sequence to the list
    
    info = SummaryInfo(sequences_lineage)
    consensus_sequence =  info.gap_consensus(
    threshold=0.95, 
    ambiguous='N')
        
    nuc_position = []
    reference_nuc = []
    lineage_nuc = []

    for i in range(len(consensus_sequence)):
        if reference_sequence[i] != "N" and consensus_sequence[i] != "N":
            if consensus_sequence[i] != reference_sequence[i]:
                nuc_position.append(i)
                reference_nuc.append(reference_sequence[i])
                lineage_nuc.append(consensus_sequence[i])
    
    defining_mutations = pd.DataFrame(
    {'nucleotide_position': nuc_position,
     'reference_nucleotide': reference_nuc,
     'lineage_nucleotide': lineage_nuc
    })
    
    lineage_index =  Bio.Align.MultipleSeqAlignment([])
    
    for sequence in designation_index: ## for a sequence in the indexed sequences
        if sequence in sequence_ID_list: ## is this sequence in the ID list? 
            lineage_index.append(designation_index[sequence]) ## if it is, add the sequence to the list    
    
    mutcount_list = [] # mutation counts 
    epi_ID = [] # sequence ID

    for sequence in lineage_index: 
        epi_ID.append(sequence.name)

        mutation_count = 0

        for index, row in defining_mutations.iterrows():
            if row["lineage_nucleotide"] == sequence.seq[row['nucleotide_position']]:
                mutation_count  += 1
    
        mutcount_list.append(str(mutation_count))    
    
    lineage_column = [lineage_of_interest] * len(mutcount_list)
    def_mut_column = [str(len(defining_mutations))] * len(mutcount_list)
    
    mutation_list = []
        
    for i in range(len(consensus_sequence)):
        if reference_sequence[i] != "N" and consensus_sequence[i] != "N":
            if consensus_sequence[i] != reference_sequence[i]:
                
                mutation_list.append(str(reference_sequence[i]) +  str(i) +str(consensus_sequence[i]))
                
        
    
    return(consensus_sequence, epi_ID, lineage_column, def_mut_column, mutcount_list, defining_mutations, mutation_list)


## this code returns the lists rather than the dataframes

code returns 7 things: 

- [0] consensus sequence for given lineage
- [1] epi ID - list of sequence names
- [2] lineage column - list of designated lineage
- [3] def_mut_column - list with no of lineage defining mutations for that lineage
- [4] mutcount_list - list of mutation counts for each sequence
- [5] dataframe of defining mutations for that lineage 
- [6] a list of mutations associated with that lineage


In [158]:
lineages_list = list(lineage_designations["lineage"].unique())

In [428]:
## the code below runs for any list of lineages, outputting a fasta file of consensus, a dataframe of defining mutations for each lineage and a dataframe of every designated sequences for that lineage, along with mutation counts.  

sequence_column = []
lineage_column = []
mutations_present_in_lineage = []
mutations_present_in_sequence = []

column1 = []
column2 = []
column3 = []

mutation_dictionary = {}
output = open("lineage_consensus_seqs.fasta", "w")

for lineage in lineages_list: ## lineages list could be any list of sequences. Defined above as lineage list is a complete list of every single mutation
        
    get_sequences = (full_function_mutations(lineage_of_interest = lineage, designation_index = sequences_index, reference_sequence = reference_A))
    
    sequence_names = (get_sequences)[1]
    sequence_lineages = (get_sequences)[2]
    mutations_lineage = (get_sequences)[3]
    mutations_in_seqs = (get_sequences)[4]
        
    sequence_column.extend(sequence_names)                    # add lineage sequence names to the list
    lineage_column.extend(sequence_lineages)                # add the lineage names to the list
    mutations_present_in_lineage.extend(mutations_lineage)  # add the total mutations to the list
    mutations_present_in_sequence.extend(mutations_in_seqs) # add the mutations per sequence to the list
   
    output.write(">" + lineage + "\n" + str(get_sequences[0]) + "\n") ## write the consensus sequence to the fasta file
    
    mutation_dictionary[lineage] = get_sequences[6] ## add list of mutations to the dictionary

    
    column1.append(lineage)
    column2.append('|'.join(map(str, mutation_dictionary[lineage])))
    column3.append(len(mutation_dictionary[lineage]))
    
    print(lineage)
    
## write the consensus sequences to a file

sequences = pd.DataFrame({'sequence_name': sequence_column,
     'lineage': lineage_column,
     'mutations_present_in_lineage': mutations_present_in_lineage,
     'mutations_present_in_sequence': mutations_present_in_sequence})

## this code outputs the sequences dataframe, which shows the mutation counts for every single sequence

percentage_mutations_present = []

for index, row in sequences.iterrows(): 
    if row["mutations_present_in_lineage"] == "0":   
        percentage_mutations_present.append(100)
    else:            
        percentage_mutations_present.append(round(100*int(row["mutations_present_in_sequence"])/int(row["mutations_present_in_lineage"]), 1))
    
sequences["%_mutations_present"] = percentage_mutations_present
## add a percentage mutations present column to the dataframe

# from this code, I get: 
### consensus sequences written to a single file called 
### 'sequences' dataframe - mutation counts for every sequence
### 'mutation_dictionary' - key = lineage, value = list of mutations.  

A.29
C.33
AG.1
AM.1
